<a href="https://colab.research.google.com/github/Guru4Char/Multimodal-data-processing-system/blob/main/Multimodal_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# ============================================
# STEP 1 — Install needed libraries
# ============================================
!pip install google-generativeai PyMuPDF python-docx python-pptx markdown


In [22]:
# ============================================
# STEP 2 — Import libraries
# ============================================
import google.generativeai as genai
import fitz  # PyMuPDF for PDF
from docx import Document
from pptx import Presentation
import markdown
from google.colab import files
import io


In [ ]:
# ============================================
# STEP 3 — Upload a file
# ============================================
uploaded = files.upload()

filename = list(uploaded.keys())[0]
print(f"\nFile uploaded: {filename}")


Saving Little Red Riding Hood.pdf to Little Red Riding Hood.pdf

File uploaded: Little Red Riding Hood.pdf


In [ ]:
# ============================================
# STEP 4 — Extract text depending on file type
# ============================================
def extract_text_from_pdf(path):
    text = ""
    with fitz.open(path) as pdf:
        for page in pdf:
            text += page.get_text("text")
    return text

def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs])

def extract_text_from_pptx(path):
    prs = Presentation(path)
    text_runs = []
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text_runs.append(shape.text)
    return "\n".join(text_runs)

def extract_text_from_md(path):
    with open(path, "r", encoding="utf-8") as f:
        md_text = f.read()
    html = markdown.markdown(md_text)
    return html

def extract_text_from_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

# detect type and extract
if filename.endswith(".pdf"):
    knowledge_base = extract_text_from_pdf(filename)
elif filename.endswith(".docx"):
    knowledge_base = extract_text_from_docx(filename)
elif filename.endswith(".pptx"):
    knowledge_base = extract_text_from_pptx(filename)
elif filename.endswith(".md"):
    knowledge_base = extract_text_from_md(filename)
elif filename.endswith(".txt"):
    knowledge_base = extract_text_from_txt(filename)
else:
    raise ValueError("Unsupported file type. Please upload pdf, docx, pptx, md, or txt.")

print("\n✅ Text extraction complete. Preview (first 10000 chars):\n")
print(knowledge_base[:10000])


✅ Text extraction complete. Preview (first 10000 chars):

Little Red Riding Hood (Simple English Story) 
  
Once upon a time, there was a little girl called Little Red Riding Hood. She wore a red cloak, so 
everyone called her by that name. 
One day, her mother said, “Please take this basket of food to Grandma. She is sick.” Little Red Riding 
Hood said, “Okay, Mommy.” 
Her mother said, “Don’t talk to strangers, and stay on the path.” 
Little Red Riding Hood walked through the forest. On the way, a big bad wolf saw her. He wanted to 
eat her but smiled and asked, “Where are you going, little girl?” 
She said, “I’m going to see my grandma. She is sick.” 
The wolf thought, “I will eat the grandmother first and then the girl.” 
He ran fast to Grandma’s house. He knocked on the door. “Who is it?” asked Grandma. 
“It’s me, Little Red Riding Hood,” said the wolf in a soft voice. 
Grandma said, “Come in.” The wolf jumped in and swallowed Grandma whole! Then he put on 
Grandma’s clothes and l

In [28]:
# Cell 5 — Install the official Google GenAI SDK (recommended)
!pip install -q google-genai


In [30]:
# Cell 6 — Fixed version for Google Colab
from google import genai
import getpass

# Ask for your API key (you get this from Google AI Studio)
API_KEY = getpass.getpass("Paste your Gemini / Google AI Studio API key here: ")

# ✅ Create the client with the API key directly
client = genai.Client(api_key=API_KEY)

print("Google GenAI client successfully configured! ✅")


Paste your Gemini / Google AI Studio API key here: ··········
Google GenAI client successfully configured! ✅


In [31]:
# Cell 7 — (Optional) Try listing available models in your account (if supported)
# NOTE: listing may not be supported in all SDK versions/accounts; if not, skip.
try:
    models = client.models.list()  # may return a list or raise if not allowed
    print("Available models (first 20 shown):")
    for m in models[:20]:
        print("-", m.get("name") or m.get("id") or m)
except Exception as e:
    print("Listing models is not supported or failed (that's ok). Error message:")
    print(e)
    print("\nYou can use a known supported model name such as 'gemini-2.5-flash' or 'gemini-1.5-pro' depending on your access. See docs: https://ai.google.dev/gemini-api/docs/quickstart")


Available models (first 20 shown):
Listing models is not supported or failed (that's ok). Error message:
'Model' object has no attribute 'get'

You can use a known supported model name such as 'gemini-2.5-flash' or 'gemini-1.5-pro' depending on your access. See docs: https://ai.google.dev/gemini-api/docs/quickstart


In [33]:
# Cell 9 — Simple local search fallback (no API required)
import re
from collections import Counter

def simple_local_search(document_text, question, top_k=5):
    # split into sentences (naive)
    sentences = re.split(r'(?<=[.!?])\s+', document_text.strip())
    # tokenise question
    qtokens = re.findall(r'\w+', question.lower())
    # score sentences by token overlap
    scores = []
    for s in sentences:
        stokens = re.findall(r'\w+', s.lower())
        common = set(qtokens) & set(stokens)
        scores.append((len(common), s))
    # sort by score
    scores.sort(key=lambda x: x[0], reverse=True)
    top = [s for score,s in scores if score>0][:top_k]
    if not top:
        return "No strong match found in the document (try rephrasing the question)."
    return "\n\n".join(top)

# Interactive loop using fallback
print("Using fallback local search. Type 'exit' to quit.\n")
while True:
    q = input("Your question: ")
    if q.strip().lower() == "exit":
        print("Session ended. 👋")
        break
    print("\nAnswer (from local search):\n")
    print(simple_local_search(knowledge_base, q))
    print("\n" + "-"*80)

Using fallback local search. Type 'exit' to quit.

Your question: what is wolf name?

Answer (from local search):

She is sick.” 
The wolf thought, “I will eat the grandmother first and then the girl.” 
He ran fast to Grandma’s house.

Little Red Riding Hood arrived and saw her “grandmother.” She said, “Grandma, what big eyes you 
have!” 
The wolf said, “All the better to see you with.” 
“Grandma, what big ears you have!” 
“All the better to hear you with.” 
“Grandma, what big teeth you have!” 
“All the better to eat you with!” said the wolf, and he jumped out of bed.

She wore a red cloak, so 
everyone called her by that name.

She is sick.” Little Red Riding 
Hood said, “Okay, Mommy.” 
Her mother said, “Don’t talk to strangers, and stay on the path.” 
Little Red Riding Hood walked through the forest.

On the way, a big bad wolf saw her.

--------------------------------------------------------------------------------
Your question: what is 5th line in the passage

Answer (from local 